In [25]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd

In [26]:
lbt_itp_id = 170

In [27]:
# %%sql -o lbt_positions

# # get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
# SELECT *
# FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
# WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-02_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-03_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-04_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-05_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-06_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-07_170_0.parquet"
# ORDER BY header_timestamp

In [28]:
# lbt_positions.to_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [29]:
lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [30]:
# min_date, max_date = ('2021-11-01', '2021-12-01')
# pch_routes = ('171', '172', '173', '174', '175')

# lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
#     # >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
#     >> select(_.calitp_itp_id, _.calitp_url_number,
#               _.date == _.service_date,
#               _.trip_key, _.trip_id, _.route_id,
#               _.is_in_service, _.calitp_extracted_at, _.calitp_deleted_at)
#     # >> filter(_.route_id.isin(pch_routes))
#     >> filter(_.calitp_itp_id == lbt_itp_id)
#     >> filter(_.date == max_date)
#     >> filter(_.is_in_service == True)
#     >> collect()
#     )

In [31]:
# lbt_trips.to_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [32]:
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [33]:
trips_positions_joined = (lbt_trips
                        >> filter(_.route_id.isin(pch_routes))
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')), on='trip_id')
                       )

In [34]:
trips_positions_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37119 entries, 0 to 37118
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   calitp_itp_id_x             37119 non-null  int64  
 1   calitp_url_number_x         37119 non-null  int64  
 2   date                        37119 non-null  object 
 3   trip_key                    37119 non-null  int64  
 4   trip_id                     37119 non-null  object 
 5   route_id                    37119 non-null  object 
 6   is_in_service               37119 non-null  bool   
 7   calitp_extracted_at         0 non-null      object 
 8   calitp_deleted_at           0 non-null      object 
 9   calitp_itp_id_y             37119 non-null  int64  
 10  calitp_url_number_y         37119 non-null  int64  
 11  entity_id                   37119 non-null  object 
 12  vehicle_id                  0 non-null      object 
 13  vehicle_timestamp           371

In [35]:
trips_positions_joined

,calitp_itp_id_x,calitp_url_number_x,date,trip_key,trip_id,route_id,is_in_service,calitp_extracted_at,calitp_deleted_at,calitp_itp_id_y,calitp_url_number_y,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,170,0,2021-12-01,6252713635054096030,9288743,173,True,None,None,170,0,2413,None,1638318603,33.768066,-118.190390,1638318606
1,170,0,2021-12-01,6252713635054096030,9288743,173,True,None,None,170,0,2413,None,1638318603,33.768066,-118.190390,1638318636
2,170,0,2021-12-01,6252713635054096030,9288743,173,True,None,None,170,0,2413,None,1638318603,33.768066,-118.190390,1638318636
3,170,0,2021-12-01,6252713635054096030,9288743,173,True,None,None,170,0,2413,None,1638318663,33.769480,-118.189280,1638318666
4,170,0,2021-12-01,6252713635054096030,9288743,173,True,None,None,170,0,2413,None,1638318663,33.769480,-118.189280,1638318696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37114,170,0,2021-12-01,6070960589846718749,9288666,172,True,None,None,170,0,2403,None,1638316796,33.767952,-118.191220,1638316836
37115,170,0,2021-12-01,6070960589846718749,9288666,172,True,None,None,170,0,2403,None,1638316796,33.767952,-118.191220,1638316836
37116,170,0,2021-12-01,6070960589846718749,9288666,172,True,None,None,170,0,2403,None,1638316859,33.767980,-118.191216,1638316866
37117,170,0,2021-12-01,6070960589846718749,9288666,172,True,None,None,170,0,2403,None,1638316875,33.767980,-118.191216,1638316896


In [36]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [37]:
trips_positions_joined.geometry = trips_positions_joined.buffer(50)

In [38]:
# list(trips_positions_joined.trip_id.unique())

In [39]:
to_map = trips_positions_joined >> filter(_.trip_id == '9288623') >> select(-_.date)

In [40]:
simple_map(to_map, 'vehicle_timestamp')

Map(center=[33.78207000022891, -118.11569000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…